# Телеграм боты


Как и Вконтакте, телеграм предоставляет разработчикам API. На [официальной странице](https://core.telegram.org/bots/api) можно почитать, какие запросы нужно отправлять к API.

Если у сервиса есть API, то, скорее всего, найдется программист, который напишет удобный модуль на питоне, который облегчит работу с этим API. Для Телеграма существует довольно много оберток, например: [python-telegram-bot](https://github.com/python-telegram-bot/python-telegram-bot), [pyTelegramBotAPI](https://github.com/eternnoir/pyTelegramBotAPI). Наш вариант последний.

## Ключ доступа

Сначала создадим приложение. Надо написать [`@BotFather`](https://telegram.me/botfather)'у комманду `/newbot`.


Проследуйте по инструкциям и получится токен доступа, какой-то такой:

`704418931:AAEtcZ*************`

Собственно теперь, когда у вас есть токен и имя бота, запросы ему можно посылать даже через браузер, то есть приложение уже готово. Давайте попробуем:

Если послать вот такой запрос, то вернется базовая информация о вашем боте:

`https://api.telegram.org/bot<your-bot-token>/getme`

```yaml
{
    "ok": true,
    "result": {
        "id": 7673324157,
        "is_bot": true,
        "first_name": "Just a Game",
        "username": "PuPuPuBrewCoffeeBot",
        "can_join_groups": true,
        "can_read_all_group_messages": false,
        "supports_inline_queries": false,
        "can_connect_to_business": false,
        "has_main_web_app": false
        }
}
```

Помимо этого, мы еще так же можем доставать все сообщения, которые посылались нашему боту за все время работы. Тоже вернется в формате JSON.

`https://api.telegram.org/bot<your-bot-token>/getUpdates`


## Пора создавать полноценного бота

В той директории, где вы будете писать бота, нужно создать питоновский файл (например, `conf.py`) и записать туда токен:

```
TOKEN = "ваш_токен_тут"
```

Это нужно для того, чтобы не выкладывать в репозиторий свои логины, пароли и токены доступа. Чтобы ничего не стирать перед выкладыванием в репозиторий, лучше всего выносить секретные данные в отдельный файл и сделать так, чтобы этот файл игнорировался гитом:

* создать в той же папке файл `.gitignore`,
* и написать в этом файле `conf.py`,
* после этого файл будет у вас на компьютере, но никогда не запушится в репозиторий (подробнее [здесь](https://git-scm.com/docs/gitignore)).
* Проверьте перед пушем, что он точно не загрузится в удалённый репозиторий с помощью команды `git status`.

В основной же файл с программой этот файл можно будет просто импортировать строчкой `import conf`. Тогда переменная `TOKEN` будет доступна внутри основной программы как `conf.TOKEN`.

Установим модуль для работы с телеграмом

In [2]:
!pip install pyTelegramBotAPI --q

Теперь мы можем писать бота, например, в `my_bot.py`. **Напишем бот, который считает длину сообщения.**



In [4]:
import telebot  # импортируем модуль pyTelegramBotAPI
import conf  # импортируем наш секретный токен

bot = telebot.TeleBot(conf.TOKEN)  # создаем экземпляр бота

Теперь напишем обработчики сообщений (message handlers), которые будут реагировать на сообщения.

In [6]:
# этот обработчик запускает функцию send_welcome,
# когда пользователь отправляет команды /start или /help
@bot.message_handler(commands=['start', 'help'])
def send_welcome(message):
    bot.send_message(
        message.chat.id,
        'Здравствуйте! Это бот, который считает длину вашего сообщения.'
        )


# этот обработчик реагирует на любое сообщение
@bot.message_handler(func=lambda m: True)
def send_len(message):
    bot.send_message(
        message.chat.id,
        f'В вашем сообщении {len(message.text)} символов.'
        )

Внутри декоратора `@bot.message_handler(...)` могут находиться фильтры, которые описывают, на какие сообщения реагирует данная функция. Фильтры пишутся так: сначала название фильтра, затем через знак равно его значение. Бывают фильтры четырех типов:

* `content_types`, значением является массив строк, задающих тип контента $-$ текст, аудио, файл, стикер и т.д. (по умолчанию `['text']`)
* `regexp`, значением является регулярное выражение (строка)
* `commands`, значением является массив строк, задающих команды без знака `/`
* `func`, значением является какая-то функция

Что бывает, когда боту приходит сообщение, которое подходит под несколько наших фильтров, т.е. несколько разных функций? В этом случае запускается функция, которая в вашем коде написана раньше других.

Теперь просим бота бесконечно опрашивать сервера телеграма на предмет новых сообщений (как-то так: "Мне что-нибудь пришло? А сейчас пришло что-нибудь? А сейчас? Пришло? Пришло? А сейчас написали что-нибудь мне?"). Параметр `none_stop=True` говорит, что бот должен стараться не прекращать работу при возникновении каких-либо ошибок.

In [7]:
if __name__ == '__main__':
    bot.polling(none_stop=True)

Итак, у нас получился примерно вот такой бот:

![chatexample](https://github.com/KatiaKozlova/files/blob/main/57-10/tg-bot/tg_bot_1.png?raw=true)

Немного усложним наш бот. Например, разрешим пользователю на команду `/dog` получать картинку с рандомной собакой. Почему нет?

In [ ]:
@bot.message_handler(commands=['dog'])
def bop(message):
    contents = requests.get('https://random.dog/woof.json').json()
    url = contents['url']
    bot.send_photo(chat_id=message.chat.id, photo=url)

Разберемся поподробнее с `bot.polling(none_stop=True)`

Вообще вариантов получения ботом сообщений от Телеграма два:

1. опрос (буквальный перевод слова *polling*) сервера Телеграма на наличие сообщений для бота.
2. “почтовый ящик” с IP-адресом (*webhook* можно перевести как веб-ловушка), на который приходят сообщения от сервера Телеграма.

> Самая простая аналогия с реальной почтой. Пусть почта (почтовое отделение) $-$ это сервер Телеграма, а вы $-$ это ваш бот. Тогда, в первом случае (*polling*) вам приходится ходить на почту за корреспонденцией. И если хотите получать сообщения без задержек, то придется не ходить, а буквально бегать без передышек взад и вперед. Как понимаем, жить на почте в ожидании сообщений запрещено! Во втором случае вы сообщаете почтовому отделению свой домашний адрес и ждете корреспонденцию спокойно дома, попивая чай.

См. разбор [тут](https://habr.com/ru/company/ods/blog/462141/).

Пока используем *polling*, но это не оптимальное решение, так как если вы выложите ваше приложение на какой-нибудь веб-сервер и на сервере произойдет хоть какая-нибудь ошибка или дисконнект, бот перестанет работать.


### Задание

Попробуйте написать бот-попугай, который на команды `/start` и `/help` представляется и описывает, что он делает; на любое текстовое сообщение присылает в ответ его же + картинку попугая.

## Функционал

### Привязка данных к пользователю

Допустим, что мы хотим сделать бота, который запоминает фразу, а потом, по просьбе пользователя, напоминает ее ему. Чтобы решить эту задачу, нам понадобится где-то хранить последнее сообщение пользователя.

Если мы будем использовать переменную, то сможем сохранить сообщение только одного пользователя. Например, Ваня попросил запомнить слово *linguist*. Мы положили эту строку в переменную `note`. Потом Василиса попросила запомнить слово *elephant*, и мы снова положили это в переменную `note`. Когда Ваня попросит нас напомнить его последнее сообщение, мы напишем ему *elephant* вместо *linguist*. Неуспех!

Удобнее всего хранить все данные, которые привязаны к конкретному пользователю, в словаре. Ключем в этом словаре будет id пользователя, а значением $-$ произвольные данные.

Допустим, что наш словарь называется `notes`. Теперь, когда Ваня (`id1076`) пришлет слово *linguist* мы положим его в `notes[1076]`, а слово *elephant* от Василисы (`id1402`) - в `notes[1402]`. Так как теперь мы используем разные переменные для хранения слова, сообщения от разных пользователей не будут путаться.

In [8]:
import telebot
import conf

bot = telebot.TeleBot(conf.TOKEN)

#создаем словарь
notes = {}

@bot.message_handler(commands=['remind'])
def remind(message):
    user_id = message.chat.id
    if user_id not in notes:
        bot.send_message(user_id, "Вы мне еще не писали.")
    else:
        bot.send_message(user_id, notes[user_id])

@bot.message_handler(content_types=['text'])
def remember(message):
    user_id = message.chat.id
    notes[user_id] = message.text
    bot.send_message(user_id, "Я запомнил")

bot.polling(none_stop=True)

#### Задание

Перепешите код выше так, чтобы бот мог запоминать несколько слов от пользователя, но при просьбе напоминания возвращал ему самое первое в очереди.

Реализуйте это при помощи дека ([`collections.deque`](https://docs.python.org/3/library/collections.html#deque-objects)).

### Кнопки

Как добавить несколько кнопок к сообщению и реагировать на их нажатия?

In [10]:
import telebot
from telebot import types

import conf

bot = telebot.TeleBot(conf.TOKEN)

@bot.message_handler(commands=["start"])
def repeat_all_messages(message):
    # создаем клавиатуру
    keyboard = types.InlineKeyboardMarkup()

    # добавляем на нее две кнопки
    button1 = types.InlineKeyboardButton(
        text="Кнопка 1",
        callback_data="button1"
        )
    button2 = types.InlineKeyboardButton(
        text="Кнопка 2",
        callback_data="button2"
        )
    keyboard.add(button1)
    keyboard.add(button2)

    # отправляем сообщение пользователю
    bot.send_message(
        message.chat.id,
        "Нажмите кнопку!",
        reply_markup=keyboard
        )

# функция запустится, когда пользователь нажмет на кнопку
@bot.callback_query_handler(func=lambda call: True)
def callback_inline(call):
    if call.message:
        if call.data == "button1":
            bot.send_message(
                call.message.chat.id,
                "Вы нажали на первую кнопку."
                )
        if call.data == "button2":
            bot.send_message(
                call.message.chat.id,
                "Вы нажали на вторую кнопку."
                )

bot.polling(none_stop=True)

Можно задать параметры клавиатуры, например, количество рядов кнопок, ширину рядов:

```python
markup = types.ReplyKeyboardMarkup(row_width=2) # по умолчанию 3
itembtna = types.KeyboardButton('a')
itembtnv = types.KeyboardButton('b')
itembtnc = types.KeyboardButton('c')
itembtnd = types.KeyboardButton('d')
itembtne = types.KeyboardButton('e')

markup.row(itembtna, itembtnv)
markup.row(itembtnc, itembtnd, itembtne)

tb.send_message(
    chat_id,
    "Choose one letter:",
    reply_markup=markup
    )
```



Скрывает присланную клавиатуру:

```python
markup = types.ReplyKeyboardRemove()
tb.send_message(
    chat_id,
    message,
    reply_markup=markup
    )
```



Заставляет пользователя ответить на сообщение:
```python
markup = types.ForceReply()
tb.send_message(
    chat_id,
    "Send me another word:",
    reply_markup=markup
    )
```



### Какие еще есть функции?

In [ ]:
# sendMessage
tb.send_message(chat_id, text)

# editMessageText
tb.edit_message_text(new_text, chat_id, message_id)

# forwardMessage
tb.forward_message(to_chat_id, from_chat_id, message_id)

# All send_xyz functions which can take a file as an argument,
# can also take a file_id instead of a file.
# sendPhoto
photo = open('/tmp/photo.png', 'rb')
tb.send_photo(chat_id, photo)
tb.send_photo(chat_id, "FILEID")

# sendAudio
audio = open('/tmp/audio.mp3', 'rb')
tb.send_audio(chat_id, audio)
tb.send_audio(chat_id, "FILEID")

## sendAudio with duration, performer and title.
tb.send_audio(CHAT_ID, file_data, 1, 'eternnoir', 'pyTelegram')

# sendVoice
voice = open('/tmp/voice.ogg', 'rb')
tb.send_voice(chat_id, voice)
tb.send_voice(chat_id, "FILEID")

# sendDocument
doc = open('/tmp/file.txt', 'rb')
tb.send_document(chat_id, doc)
tb.send_document(chat_id, "FILEID")

# sendSticker
sti = open('/tmp/sti.webp', 'rb')
tb.send_sticker(chat_id, sti)
tb.send_sticker(chat_id, "FILEID")

# sendVideo
video = open('/tmp/video.mp4', 'rb')
tb.send_video(chat_id, video)
tb.send_video(chat_id, "FILEID")

# sendVideoNote
videonote = open('/tmp/videonote.mp4', 'rb')
tb.send_video_note(chat_id, videonote)
tb.send_video_note(chat_id, "FILEID")

# sendLocation
tb.send_location(chat_id, lat, lon)

# sendChatAction
# action_string can be one of the following strings:
# 'typing', 'upload_photo', 'record_video', 'upload_video',
# 'record_audio', 'upload_audio', 'upload_document' or 'find_location'.
tb.send_chat_action(chat_id, action_string)

# getFile
# Downloading a file is straightforward
# Returns a File object
import requests
file_info = tb.get_file(file_id)

file = requests.get(f'https://api.telegram.org/file/bot{conf.TOKEN}/{file_info.file_path}')

### Как отправить и как скачать стикер?

Сначала нужно узнать ID стикера. Например, можно отправить любой стикер боту [`@idstickerbot`](https://telegram.me/idstickerbot), и он вернёт Вам ID.

In [ ]:
@bot.message_handler(content_types=["text"])
def text(message):
    if message.text == 'sticker':
        bot.send_sticker(
            message.chat.id,
            'CAACAgIAAxkBAAENqm5nmwABzztusHt8t3VbtayZvkilLjMAAkcUAALrDclLB7_h-b06JEg2BA'
            )

Если пользователь пришлёт слово `sticker`, то мы отправим ему стикер. Используем метод `send_sticker`.

А как сохранить на комп присланный стикер?

In [ ]:
@bot.message_handler(content_types=["sticker"])
def handle_docs_audio(message):
    # получаем ID стикера
    sticker_id = message.sticker.file_id
    # получаем путь, где лежит файл стикера на сервере телеграма
    file_info = bot.get_file(sticker_id)
    # скачиваем файл
    urllib.request.urlretrieve(
        f'http://api.telegram.org/file/bot{conf.TOKEN}/{file_info.file_path}',
        file_info.file_path
        )

Простой бот, который отвечает текстом или стикером, в зависимости от послания пользователя:

In [11]:
import telebot
import conf

bot = telebot.TeleBot(conf.TOKEN)

@bot.message_handler(commands=['start'])
def start_message(message):
    bot.send_message(message.chat.id, 'Привет, ты написал мне, ура!')

@bot.message_handler(content_types=['text'])
def send_text(message):
    if message.text.lower() == 'привет':
        bot.send_message(message.chat.id, 'Привет!')
    elif message.text.lower() == 'пока':
        bot.send_message(message.chat.id, 'Пока!')
    elif message.text.lower() == 'люблю программировать':
        bot.send_sticker(
            message.chat.id,
            'CAACAgIAAxkBAAENqm5nmwABzztusHt8t3VbtayZvkilLjMAAkcUAALrDclLB7_h-b06JEg2BA'
            )

@bot.message_handler(content_types=['sticker'])
def sticker_id(message):
    print(message)

bot.polling()

### Как отправить длинное текстовое сообщение?

Если Вы хотите отправить сообщение, где больше 5000 символов, его придется разбить

In [ ]:
from telebot import util

large_text = open("large_text.txt", "rb").read()

# разобьем текст на 3000 символов
# split_string возвращает список разбитых кусочков текста
splitted_text = util.split_string(large_text, 3000)
for text in splitted_text:
	bot.send_message(message.chat.id, text)

### Как обратиться к пользователю по имени?

Объект `message` содержит ин-фу о пользователе; чтобы получить имя, нужно использовать `message.from_user.first_name`

In [12]:
import telebot
import conf

bot = telebot.TeleBot(conf.TOKEN)

@bot.message_handler(commands=['start'])
def send_welcome(message):
    bot.reply_to(message, f"Привет, {message.from_user.first_name}!")

bot.polling()

Чтобы узнать больше опций, можно просто распечатать:

In [ ]:
def send_welcome(message):
    print(message.from_user)

### Ещё чуть-чуть про `polling`

- `none_stop`: boolean (`default=False`) $-$ не прекращай запрашивать, пока не получишь от серверов Телеграма ошибку.
- `interval`: boolean (`default=False`) $-$ интервал между запросами, модификация приводит к замедлению реакции бота.
- `timeout`: integer (`default=20`) $-$ промежутки в секундах.


In [13]:
bot.polling(none_stop=False, interval=False, timeout=20)

# Марковские цепи

> **Цепь Маркова** *(англ. Markov chain)* — последовательность случайных событий с конечным или счётным числом исходов, характеризующаяся тем, что при фиксированном настоящем будущее независимо от прошлого. Процесс в каждый момент времени находится в одном из состояний.

[Вот хороший тьюториал](https://tproger.ru/translations/markov-chains/), где подробно (с картинками!) описан процесс работы этого алгоритма и дан пример написания генератора текста на основе цепи Маркова с нуля. В этом полезно разобраться на досуге, хотя все уже сделано за нас. Есть несколько питоновских библиотек для генерации текста с помощью марковских цепей — например, [`markovify`](https://github.com/jsvine/markovify).

In [ ]:
!pip install markovify --q

In [25]:
!wget https://raw.githubusercontent.com/KatiaKozlova/files/refs/heads/main/57-10/tg-bot/citations.txt

--2025-01-30 05:37:03--  https://raw.githubusercontent.com/KatiaKozlova/files/refs/heads/main/57-10/tg-bot/citations.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 46900 (46K) [text/plain]
Saving to: ‘citations.txt’

citations.txt       100%[===================>]  45.80K  --.-KB/s    in 0.01s   

2025-01-30 05:37:03 (3.88 MB/s) - ‘citations.txt’ saved [46900/46900]



In [17]:
with open('citations.txt', encoding='utf-8') as citations_file:
    train = citations_file.read()

In [18]:
train[:200]

'“Dogs have Owners, Cats have Staff.”\n“I do not fear computers. I fear lack of them.” – Isaac Asimov\n“A computer once beat me at chess, but it was no match for me at kick boxing.” – Emo Philips\n“Comput'

In [19]:
import markovify

m = markovify.Text(train)

In [22]:
for i in range(5):
    print(m.make_sentence())

Therefore, if you write the code in the way of your work.
~Albert W. Atwood When solving problems, dig at the roots instead of how little he can give for a dollar, instead of just hacking at the roots instead of how little he can give for a dollar, is bound to succeed.
~Bruce Hamilton If you don't have time to do it right you must give up before you get it right.
~Abraham Lincoln It is more than probable that the only form of life we have never done before.
~Peter F. Drucker Continuous improvement is all about.


In [23]:
for i in range(5):
    print(m.make_short_sentence(max_chars=100))

~Al Diamond One half of knowing what you must give up before you get it.
~Peter F. Drucker ~ The world we have created so far is purely destructive.
~Author Unknown It is the specific instrument of entrepreneurship.
~Ken Stork Everything can be a lean process.
It is more than probable that the parts you are reassembling were disassembled by you.


#### Задание

Напишите простого бота, который присылет пользователю сгенерированную с помощью марковской цепи ([`markovify`](https://github.com/jsvine/markovify)) фразу. Пользователь может ввести желаемую длину фразы, а если он этого не сделал, то шлем фразу произвольной длины. Обучить марковскую цепь можно на чем угодно. Чем больше будет объем обучающих данных, тем интереснее будут результаты! А чтобы было не скучно, можно смешать несколько разных по тематике и стилю текстов.